In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from colorama import Fore, Back, Style
from tqdm import tqdm_notebook as tqdm
from ipywidgets import Button, Layout
from IPython.display import HTML
import base64
output = widgets.Output()
newseq = []
newnewseq = []

In [ ]:
%%html
<h1><u><span style="color:purple">  THE PROTEIN TRUNCATOR TOOL </span></h1> </u>

In [ ]:
%%html
<ul>
<li><b> <span style="color:purple"> PLEASE INPUT ONE SEQUENCE </b> </span> </li>
<li><b> <span style="color:purple"> THE TOOL IS NOT SENSITIVE TO CAPITALIZATION. YOU CAN INPUT AHGKL or ahgkl. TOOL WILL UNDERSTAND.   </b> </span> </li>
<li><b> <span style="color:purple"> THE TOOL ONLY UNDERSTANDS ONE LETTER AMINO ACID CODE </b> </span> </li>
<li><b> <span style="color:purple"> ALL SPACES BETWEEN THE AMINO ACIDS WILL BE REMOVED </b> </span> </li>
<li><b> <span style="color:purple"> WE WILL TELL YOU IF WE DETECT ANY NON-AMINO ACID SYMBOL. </b> </span> </li>
<li><b> <span style="color:purple"> ALL NON-AMINO ACID SYMBOLS ARE AUTOMATICALLY REMOVED. e.g HGJK becomes HGK </b> </span> </li>
</ul>

In [ ]:
molecular_weight = {
    'A': 71.0788,
    'E': 129.1155,
    'R': 156.1875,
    'N': 114.1038,
    'D': 115.0886,
    'C': 103.1388, 
    'Q': 128.1307, 
    'G': 57.0519,
    'H': 137.1411,
    'I': 113.1594,
    'L': 113.1594,
    'M': 131.1926,
    'F': 147.1766,
    'P': 97.1167,
    'S': 87.0782,
    'T': 101.1051,
    'W': 186.2132,
    'Y': 163.1760,
    'V': 99.1326,
    'K': 128.1741, 
}
allowed_aa_list = ["A", "R", "N", "D", "C", "Q", "E", "G","H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

In [ ]:
@output.capture(clear_output=False,wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def savesequence(b):
    p=0
    weight = 18.01524
    filename = 'results.csv'
    # weight = 18.01524
    title = "Click here to download"
    name = str(nam.value)
    sequences = str(sequen.value)
    Peak = float(your_peak.value)
    prop = int(y.value)
    cutofv = int(Peak/186.2132)-3
    maxim = int(Peak/57.0519)+3
    sequences = sequences.upper()
    sequences = list(sequences.split(";"))
    for sequence in sequences:
        x = sequence.replace(' ', '')
        newseq.append(x)

    for sequence in newseq:
        if sequence == '':
            newseq.remove('')
    for sequence in newseq:
        p+=1
        for aa in sequence:
            if aa not in allowed_aa_list:
                print(Fore.MAGENTA + Style.BRIGHT)
                print(f"in the sequence {p} we found non-amino acid symbol: {aa}")
                print("we removed it")
                sequence = sequence.replace(aa, "")
        newnewseq.append(sequence)
    #print(newnewseq)
    results = [['Result sequence', 'MW_Da','Start', 'End', 'Target name']]

# So that we can have more than one sequence

    for sequence in newnewseq:
        for aa_position in tqdm(range(0, len(sequence)+1)):
            for i in range(aa_position, len(sequence)+1):
                #positions = (aa_position+1, i)
                position1 = aa_position+1
                position2 = i
                fragment = sequence[aa_position:i]
                #fragment_start = fragment[0]
                #fragment_end = fragment[-1]
                #pos_sum = f"{fragment_start}{position1} - {fragment_end}{position2}"
                if len(fragment) >= cutofv and len(fragment) <= maxim:
                    weight = 18.01524
                    for aa in fragment:
                        
                        weight += molecular_weight[aa]
                        
                #print(fragment, weight)
                
                # Saving the results
                    results.append([fragment, weight,position1, position2, name])
    final = pd.DataFrame(results).T.set_index(0).T
    final['Difference from peak'] = abs(Peak-final['MW_Da'])
    final_sorted = final.sort_values('Difference from peak', ascending = True)
    very_final_sorted = final_sorted[final_sorted['Difference from peak'] <= prop]
    #print(df)
    very_final_sorted.to_csv('results.csv')
    display(very_final_sorted.head(50))
    display(very_final_sorted.tail(3))
    
   # def create_download_link(filename, title = "Click here to download: "):  
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    
    html = '<a download="results.csv" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    display(HTML(html))
print("Input your sequence in the window below")
sequen = widgets.Text(placeholder='Input your sequence please')
display(sequen)
print("Input mass of your peak")
your_peak = widgets.Text(placeholder='Input the mass of your observed fragment')
display(your_peak)
print("Input the name of your target")
nam = widgets.Text(placeholder = 'Input the name of the target')
display(nam)
print("Set your maximum difference from peak in daltons (Da)... Default is 10 Da")
print("Maximum is 1000 Da ""(1kDa)")
dif = widgets.FloatText()
y = widgets.IntSlider(value = 10, min=1, max=1000, step=1,description='Difference', disabled=False, continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(dif, y)
mylink = widgets.jslink((y, 'value'), (dif, 'value'))
run_button = widgets.Button(
      description = 'Submit your sequence',layout=Layout(width='18%', height='45px', border='5px solid purple')
)

run_button.on_click(savesequence)
display(run_button)
output

In [ ]:
%%html 
<span style ="color:purple"> To start from scratch refresh the window </span>